### **🧩 Importing all necessary libraries and Paths**

In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Paths
train_dir = 'train'  # adjust to your dataset folder
test_dir = 'test'    # test folder with images

# Image size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

### EfficientNetB1

In [2]:
# ======================
# 🧠 EfficientNetB1 - Frozen
# ======================

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training')
val_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation')

Found 560 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [3]:
base_model_b1 = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model_b1.trainable = False

x = GlobalAveragePooling2D()(base_model_b1.output)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)
model_b1 = Model(inputs=base_model_b1.input, outputs=output)

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [4]:
model_b1.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
history_b1 = model_b1.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 34s 656ms/step - accuracy: 0.5070 - loss: 0.7038 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 20s 565ms/step - accuracy: 0.5082 - loss: 0.6952 - val_accuracy: 0.5000 - val_loss: 0.6941
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 554ms/step - accuracy: 0.5344 - loss: 0.6949 - val_accuracy: 0.3143 - val_loss: 0.6940
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 555ms/step - accuracy: 0.4759 - loss: 0.7026 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 549ms/step - accuracy: 0.5086 - loss: 0.6951 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 546ms/step - accuracy: 0.5124 - loss: 0.6992 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 542ms/step - accuracy: 0.4773 - loss: 0.7017 - val_accuracy: 0.5000 - val_loss: 0.6941
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 540ms/step - accuracy: 0.5178 - loss: 0.6928 - val_accuracy: 0.500

In [5]:
from sklearn.metrics import accuracy_score
import numpy as np
# Get validation predictions
val_generator.reset()
val_preds = model_b1.predict(val_generator)
val_preds_classes = (val_preds > 0.5).astype(int)

# True labels
true_labels = val_generator.classes

# Compute accuracy
val_accuracy = accuracy_score(true_labels, val_preds_classes)
print(f"✅ Validation Accuracy: {val_accuracy:.4f}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 793ms/step
✅ Validation Accuracy: 0.5071


In [6]:
# Predict test set
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=1, class_mode=None, shuffle=False)
preds_b0 = model_b1.predict(test_gen)
results_b0 = pd.DataFrame({
    'image file': [os.path.basename(f) for f in test_gen.filenames],
    'label': ['M' if p > 0.5 else 'N' for p in preds_b0]
})
results_b0.to_csv('EfficientNetB1.csv', index=False)
print("✅ Saved EfficientNetB0.csv")

Found 300 images belonging to 1 classes.


c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


300/300 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step
✅ Saved EfficientNetB0.csv


- The code that gave the highest score in kaggle is the one using B1 without fine tuning, i just put them in the same notebook 

### EfficientNetB1 fine tuned 

In [7]:
# Data augmentation
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
train_gen_aug = train_datagen_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training')
val_gen_aug = train_datagen_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation')

Found 560 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [8]:
base_model_b1_ft = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model_b1_ft.trainable = False

x = GlobalAveragePooling2D()(base_model_b1_ft.output)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model_b1_ft = Model(inputs=base_model_b1_ft.input, outputs=output)

model_b1_ft.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
history_b1_ft_1 = model_b1_ft.fit(train_gen_aug, validation_data=val_gen_aug, epochs=10)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 35s 705ms/step - accuracy: 0.5312 - loss: 0.6960 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 610ms/step - accuracy: 0.5353 - loss: 0.6948 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 617ms/step - accuracy: 0.5183 - loss: 0.6986 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 613ms/step - accuracy: 0.5002 - loss: 0.7087 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 625ms/step - accuracy: 0.5157 - loss: 0.6974 - val_accuracy: 0.4929 - val_loss: 0.6930
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 640ms/step - accuracy: 0.4603 - loss: 0.7065 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 633ms/step - accuracy: 0.4782 - loss: 0.7105 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 621ms/step - accuracy: 0.5372 - loss: 0.6871 - val_accu

In [9]:
# Gradual unfreezing (last 30 layers)
base_model_b1_ft.trainable = True
for layer in base_model_b1_ft.layers[:-30]:
    layer.trainable = False

model_b1_ft.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
early_stop_b1 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_sched_b1 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

history_b1_ft_2 = model_b1_ft.fit(train_gen_aug, validation_data=val_gen_aug, epochs=20,
                                  callbacks=[early_stop_b1, lr_sched_b1])

Epoch 1/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 798ms/step - accuracy: 0.5540 - loss: 0.6984 - val_accuracy: 0.5000 - val_loss: 0.6934 - learning_rate: 1.0000e-05
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 713ms/step - accuracy: 0.5487 - loss: 0.7082 - val_accuracy: 0.5000 - val_loss: 0.6940 - learning_rate: 1.0000e-05
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 714ms/step - accuracy: 0.4962 - loss: 0.7171 - val_accuracy: 0.5000 - val_loss: 0.6945 - learning_rate: 1.0000e-05
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 579ms/step - accuracy: 0.4560 - loss: 0.7289
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 708ms/step - accuracy: 0.4568 - loss: 0.7288 - val_accuracy: 0.5000 - val_loss: 0.6946 - learning_rate: 1.0000e-05
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 719ms/step - accuracy: 0.4583 - loss: 0.7270 - val_accuracy: 0.5000 - val_loss: 0.6947 - learning_rate: 5.0000e-06
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 25s 715ms/step - accuracy:

In [13]:
# Predict test set
preds_b1_ft = model_b1_ft.predict(test_gen)
results_b1_ft = pd.DataFrame({
    'image file': [os.path.basename(f) for f in test_gen.filenames],
    'label': ['M' if p > 0.5 else 'N' for p in preds_b1_ft]
})
results_b1_ft.to_csv('EfficientNetB1_Ft.csv', index=False)
print("✅ Saved EfficientNetB1_Finetuned_predictions.csv")

300/300 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step
✅ Saved EfficientNetB1_Finetuned_predictions.csv
